In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier, LGBMClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

### 결측치 제거 

In [2]:
# 데이터 가져오기
train_data = pd.read_csv("train.csv")
train_data_drop = train_data.dropna()

print(train_data_drop.info())


'''
train_data_drop = train_data.dropna()

train_drop = train_data_drop.iloc[:, 1:27]
#train_drop = train_drop.drop(['country'], axis=1)
target_drop = train_data_drop.iloc[:, 69]

'''
# main_data_drop = pd.concat([idx1to26_drop, idx41to56_drop], axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12639 entries, 0 to 14999
Data columns (total 70 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         12639 non-null  int64  
 1   Q1            12639 non-null  float64
 2   Q2            12639 non-null  float64
 3   Q3            12639 non-null  float64
 4   Q4            12639 non-null  float64
 5   Q5            12639 non-null  float64
 6   Q6            12639 non-null  float64
 7   Q7            12639 non-null  float64
 8   Q8            12639 non-null  float64
 9   Q9            12639 non-null  float64
 10  Q10           12639 non-null  float64
 11  Q11           12639 non-null  float64
 12  Q12           12639 non-null  float64
 13  Q13           12639 non-null  float64
 14  Q14           12639 non-null  float64
 15  Q15           12639 non-null  float64
 16  Q16           12639 non-null  float64
 17  Q17           12639 non-null  float64
 18  Q18           12639 non-nu

"\ntrain_data_drop = train_data.dropna()\n\ntrain_drop = train_data_drop.iloc[:, 1:27]\n#train_drop = train_drop.drop(['country'], axis=1)\ntarget_drop = train_data_drop.iloc[:, 69]\n\n"

In [3]:
data = pd.concat((train_data_drop.iloc[:,1:27], train_data_drop.iloc[:,41:57]), axis=1)
td = train_data_drop['nerdiness'].to_numpy()
td_1 = td.reshape(-1,1)

print(data.shape)
print(td_1.shape)
train_input, test_input, train_target, test_target = train_test_split(data, td_1, test_size=0.2, random_state=42)

(12639, 42)
(12639, 1)


### 랜덤 포레스트 작업

In [4]:
rf = RandomForestClassifier(n_jobs=-1)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.7664921161342875


### 특성 중요도 추가 

In [5]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.04675861 0.04434308 0.02573744 0.04035245 0.02966773 0.06235841
 0.02639391 0.03045617 0.02896488 0.02478212 0.03688391 0.03595398
 0.03121349 0.03774705 0.04875298 0.03221102 0.0267639  0.03223916
 0.03447494 0.03300057 0.03147311 0.03121608 0.02935912 0.02568233
 0.03331554 0.03200525 0.00272332 0.00493195 0.01284817 0.00218865
 0.00313696 0.00753597 0.01022077 0.01247733 0.00605144 0.00297814
 0.01201578 0.01094951 0.0098478  0.0057128  0.00277934 0.00149485]


### 하이퍼파라미터 튜닝 / 그리드 서치

In [23]:
### parameter 들을 dictionary 형태로 설정
# params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005],}

params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth':range(5,20,1), 'min_samples_split':range(2, 100, 10)}
dtree = DecisionTreeClassifier()


In [24]:
# refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.  
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가
gs.fit(train_input, train_target)

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [27]:
#best parameter 출력
print(gs.best_params_)
# GridSearchCV 결과 추출하여 DataFrame으로 변환
# scores_df = pd.DataFrame(dtree.cv_results_)
# scores_df[['params', 'mean_test_score', 'rank_test_score', \
#            'split0_test_score', 'split1_test_score', 'split2_test_score']]

print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 14, 'min_impurity_decrease': 0.0002, 'min_samples_split': 2}
0.7059638098562868


### OOB 점수 출력

In [28]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.778360201760459


### 테스트 데이터 가져오기

In [29]:
test_data = pd.read_csv("test.csv")
test_data_rpm = test_data.fillna(test_data.mean())

In [30]:
# print(test_data_rpm.shape)

test_data_input = pd.concat((test_data_rpm.iloc[:,1:27], test_data_rpm.iloc[:,41:57]), axis=1)
print(test_data_input.shape)

(35452, 42)


### 모델링_Grid Search

In [31]:
# rf.fit(train_input, train_target)
gs_pred= gs.predict(test_data_input)

### 제출 파일 생성

In [32]:
submission = pd.read_csv('sample_submission.csv')

In [34]:
submission["nerdiness"] = gs_pred
submission.to_csv("baseline.csv", index = False)